# Decorator Application

In [12]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'{run_dt}: called {fn.__name__}')
        return result
    
    return inner

In [30]:
@logged
def func1():
    x = 10
    pass

In [41]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner

In [47]:
@logged
@timed
def fact(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1,n+1))

In [48]:
fact(5)

fact ran for 0.000018s
2021-11-08 04:45:02.787760+00:00: called fact


120

In [49]:
fact(10)

fact ran for 0.000008s
2021-11-08 04:45:03.927608+00:00: called fact


3628800